In [2]:
import pandas as pd
from modules import preprocessing as pre
from modules import extract_keywords as ek
from modules import visualise as vis
import models.ovr_model as ovr
import hyperparm as hpt

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import re
import random
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from sklearn.model_selection import train_test_split
import gensim.downloader
from tqdm import tqdm

MIN_COSINE_VALUE = 0.28

def load_word2vec_model():
    with tqdm(total=1, desc="Loading Word2Vec Model", unit="model") as pbar:
        w2v_model = gensim.downloader.load('word2vec-google-news-300')
        pbar.update(1)
    return w2v_model

w2v = load_word2vec_model()

def merge_lists(l1, min_cosine_value, w2v, ref_list):
    final_list = []
    
    for word in l1:
        if word in w2v:
            similarities = []
            for ref_word in ref_list:
                if ref_word in w2v:
                    sim = w2v.similarity(word, ref_word)
                    similarities.append(sim)
            if any(sim >= min_cosine_value for sim in similarities):
                final_list.append(word)
    if 'use' in final_list: final_list.remove('use')
    elif 'also' in final_list: final_list.remove('also')
    return final_list

def better_keywords(df):
    for i in range(len(df)):
        if df.iloc[i]['STRIDE'] == 0:
            updated_list = merge_lists(df.loc[i, 'NameDesc'], MIN_COSINE_VALUE, w2v, S_final)
        elif df.iloc[i]['STRIDE'] == 1:
            updated_list = merge_lists(df.loc[i, 'NameDesc'], MIN_COSINE_VALUE, w2v, E_final)
        elif df.iloc[i]['STRIDE'] == 2:
            updated_list = merge_lists(df.loc[i, 'NameDesc'], MIN_COSINE_VALUE, w2v, D_final)
        elif df.iloc[i]['STRIDE'] == 3:
            updated_list = merge_lists(df.loc[i, 'NameDesc'], MIN_COSINE_VALUE, w2v, I_final)
        elif df.iloc[i]['STRIDE'] == 4:
            updated_list = merge_lists(df.loc[i, 'NameDesc'], MIN_COSINE_VALUE, w2v, R_final)
        else:
            updated_list = merge_lists(df.loc[i, 'NameDesc'], MIN_COSINE_VALUE, w2v, T_final)
        
        df.at[i, 'NameDesc'] = updated_list
    return df

def basic_processing(df):
    words_to_remove = ["e.g.", "code", "may", "attack", "system", "adversary", "Adversaries"]
    for word in words_to_remove:
        df['NameDesc'] = df['NameDesc'].apply(lambda x: x.replace(word, ''))
    for word in words_to_remove:
        df['NameDesc'] = df['NameDesc'].apply(lambda x: re.sub(r'\b' + re.escape(word) + r'\b', '', x))

    # df['NameDesc'] = df['NameDesc'].str.replace(r"\b(" + "|".join(words_to_remove) + r")\b", "", regex=True)
    df['NameDesc'] = df['NameDesc'].str.replace("<br><br>", "", regex=True)
    df['NameDesc'] = df['NameDesc'].str.replace("\(Citation:.*?\)", "", regex=True)
    df['NameDesc'] = df['NameDesc'].str.replace("http\S+", "", regex=True)
    df['NameDesc'] = df['NameDesc'].str.replace("  +", " ", regex=True)
    df['NameDesc'] = df['NameDesc'].str.replace("[^A-Za-z]", " ", regex=True)
    return df

def rm_stopwords(df):
    stop_words = set(stopwords.words('english'))
    df['NameDesc'] = df['NameDesc'].apply(lambda x: [word for word in x if word not in stop_words])
    # print(f"Removed stopwords:\n {df.head(3).NameDesc}\n")
    return df

def lemmatize(df):
    lemmatizer = WordNetLemmatizer()
    def lemmatize_tokens(tokens):
        def get_wordnet_pos(word):
            tag = nltk.pos_tag([word])[0][1][0].upper()
            tag_dict = {"J": wordnet.ADJ,
                        "N": wordnet.NOUN,
                        "V": wordnet.VERB,
                        "R": wordnet.ADV}
            return tag_dict.get(tag, wordnet.NOUN)
        lemmas = [lemmatizer.lemmatize(token, get_wordnet_pos(token)) for token in tokens]
        return lemmas
    df['NameDesc'] = df['NameDesc'].apply(lambda x: lemmatize_tokens(x))
    # print(f"Lemmatized words:\n {df.head(3).NameDesc}")
    return df

def text_preprocessing(df):
    basic_processing(df)
    df['NameDesc'] = df['NameDesc'].apply(lambda x: word_tokenize(x))
    rm_stopwords(df)
    lemmatize(df)

    k = random.randint(0, len(df)) # arbitary row to show that words have been removed
    print(f"Bef rm duplicates: {len(df.iloc[k]['NameDesc'])}")
    df['NameDesc'] = df['NameDesc'].apply(lambda x: list(set([word.lower() for word in x]))) # to remove duplicates
    print(f"Aft rm duplicates: {len(df.iloc[k]['NameDesc'])}")
    print(f"Removed duplicates:\n {df.head(3).NameDesc}")

    print("=========================================")
    return df

def change_label(df):
    stride_mapping = { # STRIDE
        1: 1, # E
        10: 2, # D
        100: 3, # I
        1000: 4, # R
        10000: 5, # T
        100000: 0, # S
        11000: 4,
        100100: 0
    }
    df['STRIDE'] = df['STRIDE'].map(stride_mapping)

def split_data(df, train_set_size, test_set_size):
    '''
    train_set_size + test_set_size + dev_set_size = 1
    '''
    while True:
        df_train, temp = train_test_split(df, test_size=1-train_set_size)
        df_test, df_dev = train_test_split(temp, test_size=test_set_size)

        change_label(df_train)
        change_label(df_test)
        change_label(df_dev)

        c = set([0, 1, 2, 3, 4, 5])
        if set(df_train['STRIDE'].unique()) != c or \
            set(df_test['STRIDE'].unique()) != c or \
            set(df_dev['STRIDE'].unique()) != c:
                continue
        else: break
    return df_train, df_test, df_dev

# keywords defined from each STRIDE category
S = ['authenticate', 'username', 'password', 'access'] #add more words
T = ['modify', 'persistent', 'database', 'alter', 'open', 'network', 'internet'] #add more words
R = ['deny', 'action', 'prove', 'non-repudiation', 'item', 'sign', 'receipt', 'receive', 'evidence', 'package', 'untrace',]
I = ['exposure', 'individual', 'access', 'file', 'granted', 'intruder', 'transit']
D = ['denial', 'service', 'dos', 'web', 'server', 'unavailable', 'unusable', 'system', 'available', 'reliable']
E = ['unprivileged', 'privileged', 'access', 'compromise', 'entire', 'system', 'elevation', 'penetrate', 'defenses', 'untrusted', 'trusted']

# words to keep (manual filtering from keywords extracted from text_preprocessing())
S_keep = ['information', 'detection', 'take',  'include', 'malicious', 'control', 'network', 'search', 'name', 'access', 'infrastructure', 'traffic', 'data', 'suspicious', 'trust', 'reconnaissance', 'email', 'phishing', 'resource', 'initial', 'visibility', 'monitor', 'server', 'form', 'open', 'potentially', 'websites', 'address', 'process', 'detect', 'credential', 'file', 'certificate', 'internet', 'install', 'key', 'online', 'link', 'source']
E_keep = ['process', 'access', 'file', 'execute', 'activity', 'execution', 'network', 'behavior', 'create', 'control', 'log', 'privilege', 'application', 'service', 'within', 'event', 'account', 'modify', 'run', 'abuse', 'monitoring', 'environment', 'binary', 'credential', 'enable', 'api', 'exe', 'function', 'payload', 'target', 'method', 'services', 'launch', 'root', 'os', 'many''accounts']
D_keep = ['service', 'target', 'tool', 'command', 'cause', 'server', 'network', 'outside', 'denial', 'dos', 'availability', 'high', 'destruction', 'infrastructure']
I_keep = ['data', 'network', 'activity', 'access', 'behavior', 'environment', 'process', 'detection', 'remote', 'base', 'target', 'tool', 'file', 'api', 'traffic', 'acquire', 'application', 'host', 'infrastructure', 'device']
R_keep = ['user', 'application', 'api', 'activity', 'audit', 'source', 'system', 'native', 'hide', 'error', 'intrusion', 'function', 'record', 'clear', 'gcp', 'permission', 'analysis', 'collection', 'updatesink', 'indicate', 'detection', 'data', 'collect', 'environment', 'call', 'limit', 'cloudtrail', 'loss', 'conduct', 'prior', 'delete', 'cloud', 'configservicev', 'cloudwatch', 'diagnostic', 'capability', 'sufficient', 'insight', 'avoid']
T_keep = ['malicious', 'file', 'activity', 'process', 'execute', 'access', 'information', 'control', 'software', 'modify', 'network', 'data', 'abuse', 'exe', 'manipulate', 'bypass', 'malware', 'functionality', 'integrity', 'dll', 'anomaly', 'install']

# combine both lists together by group
S_final = S + S_keep
T_final = T + T_keep
R_final = R + R_keep
I_final = I + I_keep
D_final = D + D_keep
E_final = E + E_keep

/var/folders/y3/qz4wgjts6dj2m3kdg2nq67200000gn/T/ipykernel_24476/1592899246.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chufeng/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/chufeng/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/chufeng/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/chuf

In [4]:
tfidf_vectorizer = TfidfVectorizer()
base_classifier = LogisticRegression()

clf = OneVsRestClassifier(base_classifier)

def vectorize(df_train, df_test):
    df_train['NameDesc'] = df_train['NameDesc'].apply(lambda x: ' '.join(x))
    df_test['NameDesc'] = df_test['NameDesc'].apply(lambda x: ' '.join(x))
    # df_dev['NameDesc'] = df_dev['NameDesc'].apply(lambda x: ' '.join(x))

    X_train_tfidf = tfidf_vectorizer.fit_transform(df_train['NameDesc']).toarray()
    X_test_tfidf = tfidf_vectorizer.transform(df_test['NameDesc']).toarray()
    # X_val_tfidf = tfidf_vectorizer.transform(df_dev['NameDesc']).toarray()

    y_train = df_train['STRIDE'].values
    y_test = df_test['STRIDE'].values
    # y_val = df_dev['STRIDE'].values

    return X_train_tfidf, X_test_tfidf, y_train, y_test

def train_loop(X_train_tfidf, X_test_tdift, y_train, y_test):
    clf.fit(X_train_tfidf, y_train)

    y_pred = clf.predict(X_train_tfidf)
    acc = accuracy_score(y_test, y_pred)
    print("Accuracy:", acc)
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    return y_pred, acc

In [11]:
from sklearn.model_selection import train_test_split

def split_data2(df, train_set_size):
    '''
    train_set_size + test_set_size + dev_set_size = 1
    '''
    while True:
        df_train, df_test = train_test_split(df, test_size=1 - train_set_size)
        
        # Ensure both df_train and df_test have the same number of samples
        # min_samples = min(len(df_train), len(df_test))
        # df_train = df_train.sample(n=min_samples)
        # df_test = df_test.sample(n=min_samples)

        change_label(df_train)
        change_label(df_test)

        c = set([0, 1, 2, 3, 4, 5])
        if set(df_train['STRIDE'].unique()) != c or set(df_test['STRIDE'].unique()) != c:
            continue
        else:
            break

    return df_train, df_test

In [37]:
PATH = '../data/external/mitre-classified.xlsx'
df = pd.read_excel(PATH)

# train test dev split
df_train, df_test = split_data2(df, train_set_size=0.3)

col_toDrop = ['Ref', 'Name', 'Desc', 'Confidentiality', 'Integrity', 'Availability', 'Ease Of Exploitation', 'References', 'Unnamed: 0']
df_train = df_train.reset_index(drop=True).drop(columns=col_toDrop)
df_test = df_test.reset_index(drop=True).drop(columns=col_toDrop)
# df_dev = df_dev.reset_index(drop=True).drop(columns=col_toDrop)
print("Data split:")
print(f"df_train:\n{df_train['STRIDE'].value_counts()}\n")
# print(f"df_dev:\n{df_dev['STRIDE'].value_counts()}\n")
print(f"df_test:\n{df_test['STRIDE'].value_counts()}")
print("=========================================\n")


# trivially extract keywords
df_train = text_preprocessing(df_train)
df_test = text_preprocessing(df_test)
# df_dev = text_preprocessing(df_dev)
print("Trivial text preprocessing:")
print(f"df_train:\n{df_train.head(1)}\n")

# obtain better set of keywords
df_train = better_keywords(df_train)

X_train_tfidf, X_test_tfidf, y_train, y_test = vectorize(df_train, df_test)

Data split:
df_train:
STRIDE
1    62
3    40
5    28
0    13
4     1
2     1
Name: count, dtype: int64

df_test:
STRIDE
1    154
3    113
5     38
0     21
2     10
4      4
Name: count, dtype: int64

Bef rm duplicates: 46
Aft rm duplicates: 36
Removed duplicates:
 0    [associate, media, rate, organization, reconna...
1    [associate, protect, wfp, protocol, advantage,...
2    [network, success, across, protocol, event, co...
Name: NameDesc, dtype: object
Bef rm duplicates: 154
Aft rm duplicates: 103
Removed duplicates:
 0    [exist, associate, web, organization, control,...
1    [masked, network, sequence, order, commonly, r...
2    [outdated, control, network, support, interpre...
Name: NameDesc, dtype: object
Trivial text preprocessing:
df_train:
                                            NameDesc  STRIDE
0  [associate, media, rate, organization, reconna...       0



In [38]:
print("Shapes after vectorization:")
print("X_train_tfidf shape:", X_train_tfidf.shape)
print("X_test_tfidf shape:", X_test_tfidf.shape)
# print("X_val_tfidf shape:", X_val_tfidf.shape)    
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)
# print("y_val shape:", y_val.shape)

Shapes after vectorization:
X_train_tfidf shape: (145, 1324)
X_test_tfidf shape: (340, 1324)
y_train shape: (145,)
y_test shape: (340,)


In [39]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB().fit(X_train_tfidf, y_train)
y_pred = clf.predict(X_test_tfidf)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("\nConfusion Matrix:")
print(conf_matrix)

Accuracy: 0.5588235294117647
Precision: 0.4857335640138408
Recall: 0.5588235294117647
F1-score: 0.46375097449967534

Confusion Matrix:
[[  0  15   0   5   0   1]
 [  0 153   0   1   0   0]
 [  0   5   0   5   0   0]
 [  0  76   0  37   0   0]
 [  0   4   0   0   0   0]
 [  0  36   0   2   0   0]]


/Users/chufeng/Documents/IARCS internship/code/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [40]:
from sklearn.ensemble import RandomForestClassifier

clf_rf = RandomForestClassifier()
clf_rf.fit(X_train_tfidf, y_train)

y_pred_rf = clf_rf.predict(X_test_tfidf)

accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf, average='weighted')
recall_rf = recall_score(y_test, y_pred_rf, average='weighted')
f1_rf = f1_score(y_test, y_pred_rf, average='weighted')
conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)

print("Random Forest Classifier Results:")
print("Accuracy:", accuracy_rf)
print("Precision:", precision_rf)
print("Recall:", recall_rf)
print("F1-score:", f1_rf)
print("\nConfusion Matrix:")
print(conf_matrix_rf)

Random Forest Classifier Results:
Accuracy: 0.5470588235294118
Precision: 0.6052904539481165
Recall: 0.5470588235294118
F1-score: 0.4652576015371067

Confusion Matrix:
[[  5  16   0   0   0   0]
 [  0 151   0   2   0   1]
 [  0   8   0   2   0   0]
 [ 13  72   0  28   0   0]
 [  0   4   0   0   0   0]
 [  1  33   0   2   0   2]]


/Users/chufeng/Documents/IARCS internship/code/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
y_pred, acc = train_loop(X_train_tfidf, X_test_tfidf, y_train, y_test)